<a href="https://colab.research.google.com/github/alexisrisaldi/prueba/blob/master/Ciudades_WPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
pip install reverse_geocode

In [35]:
import pandas as pd
import reverse_geocode
from shapely.geometry import Point
import geopandas as gpd

In [36]:
# Leer una hoja específica llamada "Familias"
df_families = pd.read_excel('Aspire.xlsx', sheet_name='Families')

# Leer una hoja específica llamada "Members"
df_members = pd.read_excel('Aspire.xlsx', sheet_name='Members')

# Leer una hoja específica llamada "Indicators"
df_indicators = pd.read_excel('Aspire.xlsx', sheet_name='Indicators')

# Leer una hoja específica llamada "Priorities"
df_priorities = pd.read_excel('Aspire.xlsx', sheet_name='Priorities')

# Leer una hoja específica llamada "Priorities"
df_achievements = pd.read_excel('Aspire.xlsx', sheet_name='Achievements')

# Leer una hoja específica llamada "Notes"
df_notes = pd.read_excel('Aspire.xlsx', sheet_name='Notes')

# Leer una hoja específica llamada "Interventions"
df_interventions = pd.read_excel('Aspire.xlsx', sheet_name='Interventions')

In [ ]:
df_families

In [39]:
# crea un nuevo DF en base a algunas columnas especificas
df_ciudades = df_families[['Family code', 'Survey number','Family location']]

In [ ]:
df_ciudades

In [ ]:
df_ciudades['Family code'].value_counts()

In [ ]:
# ordenar valores
df_ciudades.sort_values(by=['Family code', 'Survey number'], inplace=True)


In [49]:
df_ciudades

,Family code,Survey number,Family location
112,US. S.19901003,1º,34.986454; -78.35387
113,US. S.19901003,2º,34.986454; -78.35387
2153,US.00B2F6.19840824.1453,1º,40.191643; -85.26679
2154,US.00B2F6.19840824.1453,2º,40.191505; -85.26653
1964,US.013D88.19890514.1456,1º,35.77443; -78.63136
...,...,...,...
951,US.YS.19921226,2º,34.952946; -78.50862
952,US.YS.19921226,3º,34.987835; -78.43342
953,US.YS.19921226,4º,34.987835; -78.43342
967,US.ZC.19960805,1º,35.00536; -78.312965


In [48]:
# verificar si hay nulos
df_ciudades['Family location'].isnull().sum()


np.int64(0)

In [ ]:
# Separar la columna 'Family location' en dos nuevas columnas
df_ciudades[['Latitude', 'Longitude']] = df_ciudades['Family location'].str.split(';', expand=True)

# Convertir las nuevas columnas a tipo numérico (float)
df_ciudades['Latitude'] = df_ciudades['Latitude'].astype(float)
df_ciudades['Longitude'] = df_ciudades['Longitude'].astype(float)


In [53]:
df_ciudades

,Family code,Survey number,Family location,Latitude,Longitude
112,US. S.19901003,1º,34.986454; -78.35387,34.986454,-78.353870
113,US. S.19901003,2º,34.986454; -78.35387,34.986454,-78.353870
2153,US.00B2F6.19840824.1453,1º,40.191643; -85.26679,40.191643,-85.266790
2154,US.00B2F6.19840824.1453,2º,40.191505; -85.26653,40.191505,-85.266530
1964,US.013D88.19890514.1456,1º,35.77443; -78.63136,35.774430,-78.631360
...,...,...,...,...,...
951,US.YS.19921226,2º,34.952946; -78.50862,34.952946,-78.508620
952,US.YS.19921226,3º,34.987835; -78.43342,34.987835,-78.433420
953,US.YS.19921226,4º,34.987835; -78.43342,34.987835,-78.433420
967,US.ZC.19960805,1º,35.00536; -78.312965,35.005360,-78.312965


In [54]:
#Funciones para convertir latitud y longitud en una nueva columna de ciudad y país (Asumiendo que el formato de las coordenadas sea correcto)
def get_city(row):
    coordinates = (row['Latitude'], row['Longitude'])
    result = reverse_geocode.get(coordinates)
    return result['city']

def get_country(row):
    coordinates = (row['Latitude'], row['Longitude'])
    result = reverse_geocode.get(coordinates)
    return result['country']

In [55]:
df_ciudades['Country_Ver'] = df_ciudades.apply(get_country, axis=1)
df_ciudades['City'] = df_ciudades.apply(get_city, axis=1)

<ipython-input-55-3d72d5f3dff5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ciudades['Country_Ver'] = df_ciudades.apply(get_country, axis=1)
<ipython-input-55-3d72d5f3dff5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ciudades['City'] = df_ciudades.apply(get_city, axis=1)


In [56]:
#imprimir para ver si salio bien
df_ciudades

,Family code,Survey number,Family location,Latitude,Longitude,Country_Ver,City
112,US. S.19901003,1º,34.986454; -78.35387,34.986454,-78.353870,United States,Clinton
113,US. S.19901003,2º,34.986454; -78.35387,34.986454,-78.353870,United States,Clinton
2153,US.00B2F6.19840824.1453,1º,40.191643; -85.26679,40.191643,-85.266790,United States,Parker City
2154,US.00B2F6.19840824.1453,2º,40.191505; -85.26653,40.191505,-85.266530,United States,Parker City
1964,US.013D88.19890514.1456,1º,35.77443; -78.63136,35.774430,-78.631360,United States,Raleigh
...,...,...,...,...,...,...,...
951,US.YS.19921226,2º,34.952946; -78.50862,34.952946,-78.508620,United States,Roseboro
952,US.YS.19921226,3º,34.987835; -78.43342,34.987835,-78.433420,United States,Roseboro
953,US.YS.19921226,4º,34.987835; -78.43342,34.987835,-78.433420,United States,Roseboro
967,US.ZC.19960805,1º,35.00536; -78.312965,35.005360,-78.312965,United States,Clinton


In [64]:
df_city = df_ciudades[['Family code', 'Survey number','City','Country_Ver']]

In [65]:
df_city

,Family code,Survey number,City,Country_Ver
112,US. S.19901003,1º,Clinton,United States
113,US. S.19901003,2º,Clinton,United States
2153,US.00B2F6.19840824.1453,1º,Parker City,United States
2154,US.00B2F6.19840824.1453,2º,Parker City,United States
1964,US.013D88.19890514.1456,1º,Raleigh,United States
...,...,...,...,...
951,US.YS.19921226,2º,Roseboro,United States
952,US.YS.19921226,3º,Roseboro,United States
953,US.YS.19921226,4º,Roseboro,United States
967,US.ZC.19960805,1º,Clinton,United States


In [66]:
df_resultado_families = pd.merge(
    df_families,
    df_city,
    on=['Family code', 'Survey number'],
    how='inner'   # Tipo de join: inner, left, right, outer
)

In [67]:
df_resultado_families

,Organization,Project,Family code,Created at,Survey number,Survey user,Origin,Gender,Birth date,Document type,...,Early Childhood Education,General Education Diploma,Highest level,Highest level in household,Annual income,Primary economic activity:,Assistance:,Total survey time (minutes),City,Country_Ver
0,Telamon ECFS - North Carolina,NC204 Knightdale HS Center,US.20706F.19900207.4885,2024-10-01,1º,khorton,WEB,Female,1990-02-07,Agency Case Number,...,Not applicable,2,Two-Year College Degree,Two-Year College Degree,30.000,"Employed Full-Time,Employed Part-Time",Food stamps,04:15,Chapel Hill,United States
1,Telamon ECFS - North Carolina,NC204 Knightdale HS Center,US.20706F.19900207.4885,2025-04-15,2º,khorton,WEB,Female,1990-02-07,Agency Case Number,...,Not applicable,2,Two-Year College Degree,Two-Year College Degree,30.000,"Employed Full-Time,Employed Part-Time",Food stamps,04:55,Chapel Hill,United States
2,Telamon ECFS - North Carolina,NC204 Knightdale HS Center,US.0E8757.19970822.4872,2023-10-23,1º,jjarrett,WEB,Female,1997-08-22,Agency Case Number,...,No,2,High School/GED,Two-Year College Degree,50000,Employed Full-Time,WIC,07:19,Carrboro,United States
3,Telamon ECFS - North Carolina,NC204 Knightdale HS Center,US.0E8757.19970822.4872,2024-03-13,2º,jjarrett,WEB,Female,1997-08-22,Agency Case Number,...,Yes,2,High School/GED,High School/GED,NaN,Employed Full-Time,Food stamps,10:05,Knightdale,United States
4,Telamon ECFS - Tennessee,TN201 Unicoi HS Center,US.65D007.19950718.1452,2025-04-03,1º,lmaciel,MOBILE_IOS,Female,1995-07-18,Agency Case Number,...,Yes,1,High School/GED,High School/GED,0,Unemployed,"WIC,Food stamps,Social Security",46:20,Johnson City,United States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2600,Telamon ECFS - Indiana,IN CCP,US.027CA7.19990305.1455,2024-10-04,2º,scaines,MOBILE_IOS,Female,1999-03-05,Agency Case Number,...,Yes,2,High School/GED,High School/GED,2000,Employed Full-Time,"Food stamps,WIC",03:22,Muncie,United States
2601,Telamon ECFS - North Carolina,NC213 Charles E Perry HS Center,US.KN.19980108,2023-04-24,1º,awilson,WEB,Female,1998-01-08,Agency Case Number,...,Yes,2,High School/GED,High School/GED,NaN,Employed Full-Time,"Food stamps,WIC",18:11,Roseboro,United States
2602,Telamon ECFS - North Carolina,NC213 Charles E Perry HS Center,US.KN.19980108,2023-07-10,2º,awilson,WEB,Female,1998-01-08,Agency Case Number,...,Yes,1,High School/GED,High School/GED,0,Unemployed,Food stamps,04:02,Roseboro,United States
2603,Telamon ECFS - North Carolina,NC213 Charles E Perry HS Center,US.KN.19980108,2023-12-01,3º,awilson,WEB,Female,1998-01-08,Agency Case Number,...,Yes,2,High School/GED,High School/GED,50000,"Employed Full-Time,Stay-at-Home Caregiver","Food stamps,WIC",11:50,Roseboro,United States


In [68]:
df_resultado_families.drop(columns=['Family location'], inplace=True)


In [ ]:
# Crear el archivo Excel con todas las hojas
with pd.ExcelWriter('datos_familias.xlsx', engine='xlsxwriter') as writer:
    df_resultado_families.to_excel(writer, sheet_name='Families', index=False)
    df_members.to_excel(writer, sheet_name='Members', index=False)
    df_indicators.to_excel(writer, sheet_name='Indicators', index=False)
    df_priorities.to_excel(writer, sheet_name='Priorities', index=False)
    df_notes.to_excel(writer, sheet_name='Notes', index=False)
    df_interventions.to_excel(writer, sheet_name='Interventions', index=False)
    df_achievements.to_excel(writer, sheet_name='Achievements', index=False)

In [ ]:
# Descargar el archivo en tu PC
from google.colab import files
files.download('datos_familias.xlsx')